# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'C:/Users/kai/data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived','Cabin'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [2]:
df['Age']=df['Age'].fillna(df['Age'].mode())
df['Embarked']=df['Embarked'].fillna('S')

In [3]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
mean_temp=df.groupby(['Embarked'])['Age'].mean().reset_index()
mode_temp=df.groupby(['Embarked'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
max_temp=df.groupby(['Embarked'])['Age'].max().reset_index()

temp=pd.merge(mean_temp,mode_temp,how='left',on='Embarked').reset_index()
temp=pd.merge(temp,max_temp,how='left',on='Embarked').reset_index()
temp=temp.drop(['index','level_0'],axis=1)
temp.columns=['Embarked','Age_mean','Age_mode','Age_max']
temp

,Embarked,Age_mean,Age_mode,Age_max
0,C,30.814769,24.0,71.0
1,Q,28.089286,16.0,70.5
2,S,29.519335,24.0,80.0


In [4]:
newdf=pd.merge(df,temp,how='left',on='Embarked').reset_index()
newdf

,index,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Age_mean,Age_mode,Age_max
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,29.519335,24.0,80.0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,30.814769,24.0,71.0
2,2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,29.519335,24.0,80.0
3,3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,29.519335,24.0,80.0
4,4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,29.519335,24.0,80.0
5,5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,Q,28.089286,16.0,70.5
6,6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,S,29.519335,24.0,80.0
7,7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,S,29.519335,24.0,80.0
8,8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,S,29.519335,24.0,80.0
9,9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,C,30.814769,24.0,71.0


In [5]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
old_features = []
new_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        old_features.append(feature)
        
for dtype, feature in zip(newdf.dtypes, newdf.columns):
    if dtype == 'float64' or dtype == 'int64':
        new_features.append(feature)
print(f'{len(old_features)} Numeric Features : {old_features}\n')
print(f'{len(new_features)} Numeric Features : {new_features}\n')
# 削減文字型欄位, 只剩數值型欄位
df = df[old_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
#df.head()
newdf=newdf[new_features]
newdf =newdf.fillna(-1)
newdf.head()

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

9 Numeric Features : ['index', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Age_mean', 'Age_mode', 'Age_max']



,index,Pclass,Age,SibSp,Parch,Fare,Age_mean,Age_mode,Age_max
0,0,3,22.0,1,0,7.2500,29.519335,24.0,80.0
1,1,1,38.0,1,0,71.2833,30.814769,24.0,71.0
2,2,3,26.0,0,0,7.9250,29.519335,24.0,80.0
3,3,1,35.0,1,0,53.1000,29.519335,24.0,80.0
4,4,3,35.0,0,0,8.0500,29.519335,24.0,80.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [6]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
MMEncoder=MinMaxScaler()
train_X=MMEncoder.fit_transform(df)
RE=LogisticRegression()
cross_val_score(RE, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [7]:
# 新特徵 + 邏輯斯迴歸
MMEncoder=MinMaxScaler()
train_X=MMEncoder.fit_transform(newdf)
RE=LogisticRegression()
cross_val_score(RE, train_X, train_Y, cv=5).mean()

0.6914785407550287